In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.is_available()

True

In [3]:
# Step 1: Load MNIST dataset
transform = transforms.Compose([transforms.ToTensor()])

train_dataset = datasets.MNIST(root='./data', train=True, download=False, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)
# Step 2: Data preprocessing (No specific preprocessing needed for MNIST with PyTorch)

# Step 3: Define and train Bayesian classifier
class BayesianClassifier:
    def __init__(self):
        self.class_priors = torch.zeros(10)
        self.likelihoods = torch.zeros((10, 28 * 28))
        self.feature_counts = torch.zeros((10, 28 * 28))
        self.class_counts = torch.zeros(10)

    def train(self, train_loader):
        for images, labels in train_loader:
            images = images.view(-1, 28 * 28)
            for i in range(10):
                class_images = images[labels == i]
                self.feature_counts[i] += class_images.sum(dim=0)
                self.class_counts[i] += class_images.shape[0]
        
        self.class_priors = self.class_counts / self.class_counts.sum()
        self.likelihoods = (self.feature_counts + 1) / (self.class_counts.view(-1, 1) + 2)  # Laplace smoothing

    def predict(self, images):
        images = images.view(-1, 28 * 28)
        log_priors = torch.log(self.class_priors)
        log_likelihoods = torch.log(self.likelihoods)
        log_posterior = log_priors + images @ log_likelihoods.T
        return torch.argmax(log_posterior, dim=1)

# Initialize and train the classifier
classifier = BayesianClassifier()
classifier.train(train_loader)

# Step 4: Test the model
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = classifier.predict(images)
        total += labels.size(0)
        correct += (outputs == labels).sum().item()

accuracy = correct / total
print(f'Accuracy of the Bayesian classifier on the MNIST test set: {accuracy * 100:.2f}%')


Accuracy of the Bayesian classifier on the MNIST test set: 69.03%
